## Descargar y uso de un modelo word2vec

En la siguiente celda vamos a cargar un modelo preentrenado. Este modelo ha sido entrenado con el dataset [word2vec-google-news-300](https://huggingface.co/fse/word2vec-google-news-300). Este dataset está formado por noticias de Google con alrededor de 100 mil millones de palabras, y utilizando embeddings de 300 dimensiones.

NOTA: el modelo es bastante pesado (más de un 1,6 Gb) y puede tardar varios minutos en cargar.

In [1]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


La variable *wv* es una especie de diccionario donde las claves son las palabras y los valores sus embeddings. Así, podríamos acceder al embedding de la palabra *perro* como se muestra a continuación. Podemos ver como, efectivamente, el embedding consta de 300 valores numéricos.

In [2]:
# Solo mostramos los primeros 30 valores para no ocupar toda la pantalla
wv['dog'][:30]

array([ 0.05126953, -0.02233887, -0.17285156,  0.16113281, -0.08447266,
        0.05737305,  0.05859375, -0.08251953, -0.01538086, -0.06347656,
        0.1796875 , -0.42382812, -0.02258301, -0.16601562, -0.02514648,
        0.10742188, -0.19921875,  0.15917969, -0.1875    , -0.12011719,
        0.15527344, -0.09912109,  0.14257812, -0.1640625 , -0.08935547,
        0.20019531, -0.14941406,  0.3203125 ,  0.328125  ,  0.02441406],
      dtype=float32)

In [15]:
wv['dog'].shape

(300,)

## Métricas de similitud: la distancia coseno

Para medir la similitud entre vectores de alta dimensionalidad, como es el caso, se suele utilizar la distancia coseno, que se define como uno menos el coseno del ángulo que forman dichos vectores. Además, siempre va a ser una distancia entre 0 - máxima similitud - y 1 - mínima similitud.

```
cos_dist(u,v) = 1 - cos(u,v)
```

Así, si el ángulo entre *u* y *v* es 0º, la distancia coseno será

```
cos_dist(u,v) = 1 - cos(u,v) = 1 - cos(0º) = 1 - 1 = 0
```

Sin embargo, si el ángulo entre *u* y *w* es de 90º, la distancia coseno será

```
cos_dist(u,w) = 1 - cos(u,w) = 1 - cos(90º) = 1 - 0 = 1
```

Nota: cuando tenemos tantas dimensiones, los vectores tienden a ser casi ortogonales entre sí. Así, dos vectores sin relación semántica tenderán a tener una distancia cercana a 1.


Así, la distancia coseno entre *perro* y *gato* será menor que entre *perro* y *guitarra*.

In [4]:
# Importamos la función para calcular la distancia coseno
from scipy.spatial.distance import cosine

# Medimos la distancia entre "perro" y "gato"
print("La distancia coseno entre 'perro' y 'gato' es de", cosine(wv['dog'], wv['cat']).round(2))


# Medimos la distancia entre "perro" y "guitarra"
print("La distancia coseno entre 'perro' y 'gato' es de", cosine(wv['dog'], wv['guitar']).round(2))

La distancia coseno entre 'perro' y 'gato' es de 0.24
La distancia coseno entre 'perro' y 'gato' es de 0.86


Como habíamos predicho, los embeddings correspondientes a *perro* y *gato* son much más parecidos que los correspondientes a *perro* y *guitarra*.

## Relaciones semánticas

Además, los vectores pueden representar relaciones semánticas. Por ejemplo, si tomamos el vector *rey*, extraemos el género masculino restándole el valor *hombre* y le añadimos el género femenino sumándole el vector *mujer*, obtenemos un vector muy cercano al que corresponde con la palabra *reina*.

In [5]:
cosine(wv['king'] - wv['man'] + wv['woman'], wv['queen']).round(2)

0.27

Podríamos pensar que una distancia de 0.27 no es tan pequeña. El siguiente código va a calcular las similitudes entre la palabra *reina* y el resto de 3 millones de palabras - a excepción de *reina*, claro está - para obtener el embedding más cercano.

In [6]:
def get_most_similar_word(target_word, target_embedding, verbose=False):

    most_similar_word = None
    min_dist = 99

    counter = 1

    vocab = wv.key_to_index.keys()

    len_vocab = len(vocab)

    for word in vocab:

        if counter%500000==0 and verbose:
            print(str(counter)+'/'+str(len_vocab))

        if word != target_word:

            dist = cosine(target_embedding, wv[word])

            if dist < min_dist:
                min_dist = dist
                most_similar_word = word

        counter += 1

    return min_dist, most_similar_word

In [7]:
min_dist, most_similar_word = get_most_similar_word('queen', wv['queen'], verbose=True)

500000/3000000
1000000/3000000
1500000/3000000
2000000/3000000
2500000/3000000
3000000/3000000


In [8]:
print(f"La palabra más similar a 'queen' es {most_similar_word} con una distancia coseno de {min_dist.round(2)}")

La palabra más similar a 'queen' es queens con una distancia coseno de 0.26


Vemos que la palabra con un embeding más cercano a *reina* es su plural, *reinas*, con una distancia apenas un 0.01 menor a nuestro resultado de las operaciones con los vectores *rey*, *hombre* y *mujer*, con lo que efectivamente, es una reconstrucción bastante buena del vector *reina*.

Vamos ahora a coger el vector más similar a nuestra reconstrucción del vector *reina* - sin contar el vector original, *rey* - y observaremos que, efectivamente, es *reina*.

In [9]:
min_dist, most_similar_word = get_most_similar_word('king', wv['king'] - wv['man'] + wv['woman'], verbose=True)

500000/3000000
1000000/3000000
1500000/3000000
2000000/3000000
2500000/3000000
3000000/3000000


In [10]:
print(f"La palabra más similar es {most_similar_word} con una distancia coseno de {min_dist.round(2)}")

La palabra más similar es queen con una distancia coseno de 0.27


## Sesgos en el lenguaje

Explorando este tipo de relaciones semánticas, es interesante ver ciertos sesgos en el lenguaje, por ejemplo, relacionados con el género, como se exploran en [este paper](https://arxiv.org/abs/1607.06520). Por ejemplo, si tomamos el vector de *programador*, le restamos el de *hombre* y le sumamos el de *mujer*, obtenemos que el vector más cercano es de le *ama de casa*.

In [11]:
min_dist, most_similar_word = get_most_similar_word('computer_programmer', wv['computer_programmer'] - wv['man'] + wv['woman'], verbose=True)

500000/3000000
1000000/3000000
1500000/3000000
2000000/3000000
2500000/3000000
3000000/3000000


In [12]:
print(f"La palabra más similar es {most_similar_word} con una distancia coseno de {min_dist.round(2)}")

La palabra más similar es homemaker con una distancia coseno de 0.42


En otro ejemplo, se tomaba el vector de *médico*, se restaba el de *abuelo* y se sumaba el de *abuela*, obteniendo *enfermera* como vector más cercano.

In [13]:
min_dist, most_similar_word = get_most_similar_word('doctor', wv['doctor'] - wv['grandfather'] + wv['grandmother'], verbose=True)

500000/3000000
1000000/3000000
1500000/3000000
2000000/3000000
2500000/3000000
3000000/3000000


In [14]:
print(f"La palabra más similar es {most_similar_word} con una distancia coseno de {min_dist.round(2)}")

La palabra más similar es nurse con una distancia coseno de 0.37


Esto dio lugar a técnicas de eliminación de sesgos. A grosso modo, lo que proponía el paper era tomar varios pares de palabras, de los cuales un elemento era masculino y otro femenino - *hombre* y *mujer*, *chico* y *chica*, etc -, calcular el vector de género promediando las diferencias y eliminar dicha componente de los embeddings.